In [31]:
import string
import nltk
import json
from azlyrics_scrapping import create_or_open_index
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

In [32]:
#nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))|set(stopwords.words('spanish'))

In [33]:
def clean_lyrics(lyrics):
    #stopwords = set(nltk.corpus.stopwords.words('english'))
    lyrics = " ".join([word for word in lyrics.split(" ") if word not in STOPWORDS])
    return lyrics

In [48]:
def retrieve_titles_artists_lyrics(index_path='./index'):

    index = create_or_open_index(index_path)
    all_docs = index.searcher().documents() 
    all_lyrics = list()
    all_titles = list()
    all_artists = list()
    all_urls = list()

    for doc in all_docs:
        all_titles.append(doc['title'])
        all_lyrics.append(doc['full_lyrics'])
        all_artists.append(doc['artist'])
        all_urls.append(doc['url'])
    
    return all_titles, all_lyrics, all_artists, all_urls

In [ ]:
def calculate_cos_sim(all_lyrics, all_titles, all_artists, path='similarities.json'):
    
    similarities = dict()
    all_lyrics = list(map(clean_lyrics, all_lyrics))
    vectorizer = CountVectorizer().fit_transform(all_lyrics)
    vectors = vectorizer.toarray().tolist()
    csim = cosine_similarity(vectors).tolist()
    titles_artists = list(zip(all_titles, all_artists))
    titles_artists = ["{} {}".format(title, artist) for title, artist in titles_artists]

    titles_csim =list()
    for i, sim in enumerate(csim):
        titles_csim.append(list(zip(titles_artists, sim)))
    
    for j, title_artist in enumerate(titles_artists):
        similarities[title_artist] = {tit:csim[j][k] for k, tit in enumerate(titles_artists)}

    similarities = json.dumps(similarities)
    
    with open(path, "w") as outfile:
        outfile.write(similarities)

In [70]:
def calculate_cos_sim(all_lyrics, all_urls, path='similarities.json'):
    
    similarities = dict()
    all_lyrics = list(map(clean_lyrics, all_lyrics))
    vectorizer = CountVectorizer().fit_transform(all_lyrics)
    vectors = vectorizer.toarray().tolist()
    csim = cosine_similarity(vectors).tolist()

    urls_csim =list()
    for i, sim in enumerate(csim):
        urls_csim.append(list(zip(all_urls, sim)))
    
    for j, url in enumerate(all_urls):
        similarities[url] = {u:csim[j][k] for k, u in enumerate(all_urls) if u!=url}

    similarities = json.dumps(similarities)
    
    with open(path, "w") as outfile:
        outfile.write(similarities)

In [71]:
all_titles, all_lyrics, all_artists, all_urls = retrieve_titles_artists_lyrics(index_path='./index')
#calculate_cos_sim(all_lyrics, all_titles, all_artists)
calculate_cos_sim(all_lyrics, all_urls)

In [72]:
def load_similarities(path='similarities.json'):
    with open(path, 'r') as openfile:
        dic = json.load(openfile)
    return dic

In [73]:
similarities = load_similarities()

In [74]:
botellatrasbotella = similarities['https://www.azlyrics.com/lyrics/geramx/botellatrasbotella.html']

In [75]:
import operator
max_key = max(botellatrasbotella.items(), key=operator.itemgetter(1))[0]
max_key

'https://www.azlyrics.com/lyrics/badbunny/dakiti.html'

In [78]:
index_of_song = all_urls.index('https://www.azlyrics.com/lyrics/geramx/botellatrasbotella.html')
song_title, song_lyrics = all_titles[index_of_song], all_lyrics[index_of_song]

In [79]:
song_lyrics

'botella tras botella ando tomando pa\' olvidarme de ella (pa\' olvidarme de ella) de ella de ella no más hablo en todas mis pedas (en todas mis pedas) a mis compas bien hartos traigo ya me dicen "wey ya la tienes que superar" pero yo no puedo pa\' ser sinceros yo ni quiero mejor su recuerdo me lo bebo los tragos me saben mejor así estaba pensando en llamarte y yo te miro por todas partes pero ya no nos vemos puede que lo olvidemos dos tragos y vuelvo a pensarte me aferro no quiero olvidarte sentimientos ajenos los celos no son buenos y si me llamas contesto aunque nunca va a pasar eso (aunque nunca va a pasar eso) ya mejor ni te molesto porque se que vuelvo a joderlo (a huevo que vuelvo a joderlo) y ahora que ando tomando bebida me siento triste y ya va a amanecer a veces borracho las penas se olvidan pero nunca te dejé de querer fui mal partido querida todas mis pedas marcándote al cel le puse sal a la herida cómo quisiera volver al ayer sentimental yo me pongo siempre me acuerdo de 

In [80]:
index_of_similar_song = all_urls.index(max_key)
similar_song_title, similar_song_lyrics = all_titles[index_of_similar_song], all_lyrics[index_of_similar_song]

In [81]:
similar_song_lyrics

"baby ya yo me enteré se nota cuando me ve ahí donde no has llegao sabes que yo te llevaré y dime qué quieres beber es que tú eres mi bebé y de nosotros quién va a hablar si no nos dejamos ver y a veces es dolce a veces es bulgari cuando te lo quito después de lo' party las copas de vino las libras de mari tú 'tas bien suelta y yo de safari tú mueves el culo fenomenal pa' yo devorarte como animal si no te has venido yo te vo' a esperar en mi cama y lo vo' a celebrar baby a ti no me opongo y siempre te lo pongo y si tú me tira vamo' a nadar en lo hondo si es por mí te lo pongo de septiembre hasta agosto a mí sin cojones los que digan tus amigas ya yo me enteré se nota cuando me ve ahí donde no has llegao' sabes que yo te llevaré dime qué quieres beber es que tú eres mi bebé y de nosotros quién va a hablar si no nos dejamos ver (¿me sigue?) mami me tiene' jukiao sí si fuera la urus me tuviese' parkiao dando vuelta por el condado contigo siempre arrebatao tú no eres mi señora pero toma 5 